Dataset

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-02 14:46:00--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1,06M  3,83MB/s    in 0,3s    

2024-03-02 14:46:00 (3,83 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r') as file:
    text_file = file.read()
print(text_file[:300]+ "\n\n\n"+"The length of document is "+str(len(text_file)))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


The length of document is 1115394


Tokenizer

In [3]:
characters = sorted(list(set(text_file)))
print("".join(characters), len(characters))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [4]:
#Encoder
char_to_int = {c:i for i , c in enumerate(characters)}
def encoder(text: str):
    return [char_to_int[c] for c in text]


In [5]:
#Decoder
int_to_char = {i:c for i, c in enumerate(characters)}
def decoder(ints: [int]):
    return "".join([int_to_char[i] for i in ints])

In [12]:
#Testing
print(encoder("Hello wild world"))
print(decoder(encoder("Hello wild world")))

[20, 43, 50, 50, 53, 1, 61, 47, 50, 42, 1, 61, 53, 56, 50, 42]
Hello wild world


PYTORCH DATALOADER

In [6]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
data_in_tensor = torch.tensor(encoder(text_file), dtype=torch.long)
print(data_in_tensor[:10], data_in_tensor.shape, data_in_tensor.dtype)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) torch.Size([1115394]) torch.int64


In [15]:
train_data = data_in_tensor[:int(len(data_in_tensor)*3//4)]
test_data = data_in_tensor[int(len(data_in_tensor)*3//4):]

In [11]:
sentence_length = 14
train_data[:sentence_length+1]
print(decoder(encoder(text_file[:sentence_length+1])))

First Citizen:



In [17]:
for i in range(len(train_data)//sentence_length):
    print(decoder(encoder(text_file[i*sentence_length:(i+1)*sentence_length+1])))


First Citizen:


Before we proc
ceed any furthe
er, hear me spe
eak.

All:
Spea
ak, speak.

Fir
rst Citizen:
Yo
ou are all reso
olved rather to
o die than to f
famish?

All:
R
Resolved. resol
lved.

First Ci
itizen:
First, 
 you know Caius
s Marcius is ch
hief enemy to t
the people.

Al
ll:
We know't, 
 we know't.

Fi
irst Citizen:
L
Let us kill him
m, and we'll ha
ave corn at our
r own price.
Is
s't a verdict?



All:
No more 
 talking on't; 
 let it be done
e: away, away!



Second Citize
en:
One word, g
good citizens.



First Citizen
n:
We are accou
unted poor citi
izens, the patr
ricians good.
W
What authority 
 surfeits on wo
ould relieve us
s: if they
woul
ld yield us but
t the superflui
ity, while it w
were
wholesome,
, we might gues
ss they relieve
ed us humanely;
;
but they thin
nk we are too d
dear: the leann
ness that
affli
icts us, the ob
bject of our mi
isery, is as an
n
inventory to 
 particularise 
 their abundanc
ce; our
suffera
ance is a gain 
 to them Let us
s reveng

In [18]:
example_x = train_data[:sentence_length]
example_y = train_data[1:sentence_length+1]
print((example_x), (example_y))
for i in range(sentence_length):
    print("with input ",example_x[:i+1])
    print("we expect output ", example_y[i])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) tensor([47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0])
with input  tensor([18])
we expect output  tensor(47)
with input  tensor([18, 47])
we expect output  tensor(56)
with input  tensor([18, 47, 56])
we expect output  tensor(57)
with input  tensor([18, 47, 56, 57])
we expect output  tensor(58)
with input  tensor([18, 47, 56, 57, 58])
we expect output  tensor(1)
with input  tensor([18, 47, 56, 57, 58,  1])
we expect output  tensor(15)
with input  tensor([18, 47, 56, 57, 58,  1, 15])
we expect output  tensor(47)
with input  tensor([18, 47, 56, 57, 58,  1, 15, 47])
we expect output  tensor(58)
with input  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
we expect output  tensor(47)
with input  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
we expect output  tensor(64)
with input  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64])
we expect output  tensor(43)
with input  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 4

In [19]:
batch_size = 4
num_batches = len(train_data)//(batch_size*sentence_length)
print(num_batches)

14938


In [20]:
def get_batches(training_mode):
    data = train_data if training_mode else test_data
    for i in range(num_batches):
        x = data[i*sentence_length*batch_size:(i+1)*sentence_length*batch_size]
        y = data[i*sentence_length*batch_size+1:(i+1)*sentence_length*batch_size+1]
        yield x.view(batch_size, sentence_length), y.view(batch_size, sentence_length)

temp_x, temp_y = get_batches(True).__next__()
print(temp_x.shape, temp_y.shape)
print('inputs:')
print(temp_x)
print('targets:')
print(temp_y)

torch.Size([4, 14]) torch.Size([4, 14])
inputs:
tensor([[18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10],
        [ 0, 14, 43, 44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53],
        [41, 43, 43, 42,  1, 39, 52, 63,  1, 44, 59, 56, 58, 46],
        [43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1, 57, 54]])
targets:
tensor([[47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0],
        [14, 43, 44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41],
        [43, 43, 42,  1, 39, 52, 63,  1, 44, 59, 56, 58, 46, 43],
        [56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1, 57, 54, 43]])


In [9]:
import os
import numpy as np
real_data=True
if real_data:
    dataset = 'openwebtext'
    data_dir = os.path.join('helperGPT_other/data', dataset)
    train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    def get_batch(split):
        data = train_data # note ignore split in benchmarking script
        ix = torch.randint(len(data) - sentence_length, (batch_size,))
        x = torch.stack([torch.from_numpy((data[i:i+sentence_length]).astype(np.int64)) for i in ix])
        y = torch.stack([torch.from_numpy((data[i+1:i+1+sentence_length]).astype(np.int64)) for i in ix])
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
        return x, y
else:
    torch.manual_seed(1117)
    def get_batch(training_mode):
        data = train_data if training_mode else test_data
        ix = torch.randint(len(data) - sentence_length, (batch_size,))
        x = torch.stack([data[i:i+sentence_length] for i in ix])
        y = torch.stack([data[i+1:i+sentence_length+1] for i in ix])
        return x.to(device), y.to(device)

In [12]:
xb, yb = get_batch(True)
for b in range(batch_size): # batch dimension
    for t in range(sentence_length): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

NameError: name 'batch_size' is not defined

TRAINING CODE

In [16]:
import torch.nn as nn
from torch.nn import functional as F

In [63]:


"""Tokens are not talking to eachother. """

class LanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None): # idx because want to access the 48th row if token is 48
        logits = self.token_embedding_table(idx) # B, T, C
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context 
        # purpose is to make (B, T+1) then (B, T+2) and so on
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # loss is None because targets not provided
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
model_temp = LanguageModel(vocab_size=len(characters)).to(device)
logits, loss = model_temp(xb, yb)
print(logits.shape)
print(loss) # loss is 4.78 because -ln(1/65) = 4.17

torch.Size([2048, 65])


../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [42,0,0], thread: [84,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [42,0,0], thread: [85,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [42,0,0], thread: [86,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [42,0,0], thread: [87,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [42,0,0], thread: [88,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [42,0,0], thread: [89,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [42,0,0], t

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [24]:
idx = torch.zeros((1, 1), dtype=torch.long, device=device) # as we know \n is the first token

print((model_temp.generate(idx, max_new_tokens=100)))
print(decoder(model_temp.generate(idx, max_new_tokens=100)[0].tolist()))

tensor([[ 0, 29, 26, 29, 50, 23,  7, 21, 17, 54, 37,  8,  3, 62, 38, 20, 36, 33,
         45, 29, 47,  8, 18, 35, 45, 45, 23, 51,  6,  9, 32,  5, 31, 39, 52, 26,
         31, 13, 45, 64, 34, 53, 25, 32, 11,  2, 50, 33, 26, 29, 19, 47, 24, 15,
         40, 24, 28,  8, 33,  6, 54,  3,  7, 58, 47,  8, 55, 40, 10,  4, 33, 54,
         17, 37, 22,  0, 22, 53,  7, 17, 30, 42, 27, 63, 43, 45, 63, 28, 17, 38,
         55, 56, 30, 21,  3, 62, 35, 53, 41,  8, 27]], device='cuda:0')

ql:ZYULjkln'k!N,3Tc.MJN?l!3OpnUSijT:ZKpx
So-h?PUcZEy&lApvBXl;b:u
NkNC:VjzLQ
di3J
dnQ'v.ie&Wdpz-WDdJQ


In [25]:
optimizer = torch.optim.AdamW(model_temp.parameters(), lr=1e-3)

In [26]:
for steps in range(10000):
    xb, yb = get_batch(True)
    logits, loss = model_temp(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss)
print(loss.item())

tensor(2.5775, device='cuda:0', grad_fn=<NllLossBackward0>)
2.577528476715088


In [27]:
print(decoder(model_temp.generate(idx, max_new_tokens=200)[0].tolist()))


Nethy an?
BRBathalofofithou stt art gh RKmaEOUr wse. tTo,
d thay berh,
Pir, buantinem
DUceeng mea sen the masenore;Y m,
Ve titwem; ibe INous:
AHag-herugowinqr, se oun?
NGBRI:
! y mieicepr ou Ipotldsop


SELF-ATTENTION (we want to couple the tokens wihtin the sentence length)

Query: what am I looking for
Key: what do I contain
value: Query*key gives all information. for purposes of single head, we multiply that with value (for interest of single head)

In [28]:
"""# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape"""

"# version 4: self-attention!\ntorch.manual_seed(1337)\nB,T,C = 4,8,32 # batch, time, channels\nx = torch.randn(B,T,C)\n\n# let's see a single Head perform self-attention\nhead_size = 16\nkey = nn.Linear(C, head_size, bias=False)\nquery = nn.Linear(C, head_size, bias=False)\nvalue = nn.Linear(C, head_size, bias=False)\nk = key(x)   # (B, T, 16)\nq = query(x) # (B, T, 16)\nwei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)\n\ntril = torch.tril(torch.ones(T, T))\n#wei = torch.zeros((T,T))\nwei = wei.masked_fill(tril == 0, float('-inf'))\nwei = F.softmax(wei, dim=-1)\n\nv = value(x)\nout = wei @ v\n#out = wei @ x\n\nout.shape"

Notes:

Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
Each example across batch dimension is of course processed completely independently and never "talk" to each other
In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
"self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
"Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much.
forexample, if values are 0.5, -0.2, -0.2. then the softmax will look like one-hot encoding (then it would be like agregating from a single node rather then from all)

In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
# hyperparameters
batch_size = 32
sentence_length = 64
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.1
vocab_size= len(characters)

In [19]:
class Head(nn.Module):
    """single head of self attention"""
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(sentence_length, sentence_length))) # not a parameter so we create a buffer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)  we also divide by C**-0.5 like research paper
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) so it doesnt communicate with future tokens
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [20]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [21]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [22]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # we are adding x here for residual connection
        x = x + self.ffwd(self.ln2(x))
        return x

In [23]:
class FinalGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(sentence_length, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -sentence_length:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        

In [24]:
model = FinalGPT()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.211777 M parameters


In [25]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], thread: [37,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1292: indexSelectLargeIndex: block: [98,0,0], t

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [55]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decoder(m.generate(context, max_new_tokens=2000)[0].tolist()))


embsidents onouf true, whou grousanness?

PESSLEWell:
Ahow this, it for but do me pomfled;
Whose bine be my day-both fell'd neither;
And awhom soule heard, nor from shald bid ssun of rid not!

CANTICHARD:
Ay, I have say, do my labestly of thy encrue's charld as
into theave kind they make why to I him of ,
and thou and to before they fall lounce and must,
But the face to Romile.

BROKENBURY:
Have got he will mi lest him him heavide.
She Shervand's good Citizen:
I thing you to must it ois might pattlainly an their-toCitatiol:
But herd hout'st and present, shast, thou say I so must
Shalt thee th daughterly their charges!
I reath, it conclaint, to-mined here bote me.

QUEEN MARGARETII:
Warw by this handred unto furturamed thought thep at this
This,
Sinst triume, he dost operfords, accurse scrorth'd.

HENRY BISHY:
What, cousic is of the king, be in to of fights.

BRACHARL:
EDWARD IV:
Where reper'd, quoround do--
She-reath. Your as or Was innother.

CRICHAMUMBEL:
Nor sperves go
As at Warwic

In [57]:
torch.save(m.state_dict(), 'model_weights.pth')

In [ ]:
torch.save(m, 'entire_model.pth')

ModuleNotFoundError: No module named 'tqdm'